In [1]:
!pip install roboflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.4 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=e7248ec04d3149e2f229e6a84f3ad1c81c2802124ae79764fc0894542ca4c88c
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
      Successfully uninstalled pyparsing-3.0.9
  Attempting uninstal

In [159]:
#try reading images one by one
import os
import shutil
import numpy as np
import pandas as pd
import json
import re
from roboflow import Roboflow

def cleanup():
    pattern = r"results\w+"
    # If file exists, delete it.
    for f in os.listdir(dir):
        if re.search(pattern, f):
            os.remove(os.path.join(dir, f))
    if os.path.exists(pred_dir):
        #remove if exists
        shutil.rmtree(pred_dir)
    #create folder to save predictions
    os.mkdir(pred_dir)



#initialize vehicle detection model using your api_key
def initialize(api_key):

    rf = Roboflow(api_key=api_key)
    project = rf.workspace().project("vehicle-detection-gr77r")
    model = project.version(1).model

#make predictions and save dimensions of bbox
def predict(confidence=30,overlap=30,image_dir=dir):
    pred_dir = image_dir+'/predictions'
    results = []
    confidences = []
    x_dim,y_dim = [],[]
    width,height = [],[]
    images = []

    #cleanup folder to remove results from past run
    cleanup()

    #call initialize to download model
    initialize(api_key=api_key)

    for filename in os.listdir(dir):
        if filename.endswith('JPG'):
            image = os.path.join(image_dir,filename)
            images.append(image)
            #choose confidence and overlap
            result = model.predict(image_path=image, confidence=confidence, overlap=overlap)

            #save predicted image with bounding box inside predictions folder
            result.save(f"{pred_dir}/{filename}")

            #save detected object's dimensions in each image
            x_dim.append([item['x'] for item in result.json()['predictions']])
            y_dim.append([item['y'] for item in result.json()['predictions']])
            width.append([item['width'] for item in result.json()['predictions']])
            height.append([item['height'] for item in result.json()['predictions']])
            results.append(result.json())
            try:
                confidences.append(max(item['confidence'] for item in result.json()['predictions']))
            except ValueError:
                confidences.append(0)

    #create dataframe of bbox dimensions
    pred_df = pd.DataFrame({'x':x_dim,
                            'y':y_dim,
                            'width':width,
                            'height':height,
                            'max_confidence':confidences,
                            'image_path':images})
    return pred_df,results

#output to csv and json
def output(df,results):
    df.to_csv(os.path.join(dir,"results.csv"))
    with open(os.path.join(dir,"results.json"), 'w') as outfile:
        json.dump(results, outfile)


In [160]:
#public_api_key="rf_KqTy7Mm6aXchNmgfUXw8hax6H9s1"
api_key="rGuyuyi8JwnnGmsPlUgE"
dir = '/content/drive/MyDrive/soter_imgs'

df,results = predict(image_dir=dir)

#write csv and json format outputs
#creates results.csv , results.json within specified folder
output(df,results)

loading Roboflow workspace...
loading Roboflow project...
